In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Test run

In [3]:
%%sql

SELECT COUNT(*) FROM genre;

Done.


COUNT(*)
25


## Query to retrieve most popular genre sold in U.S.

In [4]:
%%sql

WITH country AS
            (SELECT c.customer_id, country, t.name, t.genre_id, quantity
            FROM customer c JOIN invoice i ON c.customer_id = i.customer_id
            JOIN invoice_line il ON i.invoice_id = il.invoice_id
            JOIN track t ON il.track_id = t.track_id
            WHERE LOWER(c.country) = "usa"),
    track_sold_genre_wise AS(
            SELECT g.name, COUNT(*) number_of_tracks_sold,
            ROUND(CAST(SUM(quantity) AS FLOAT)/
        CAST((SELECT COUNT(*) FROM country) AS FLOAT)*100,2) percentage
        FROM country c JOIN genre g ON c.genre_id = g.genre_id
        GROUP BY g.genre_id ORDER BY 2 DESC
    )

    
SELECT * FROM track_sold_genre_wise;
    
  
            
            

Done.


name,number_of_tracks_sold,percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


According to the sales record it would be profitable to add the following genre in the collection
- Red Tone (Punk)
- Meteor and the Girls (Pop)
- Slim Jim Bites (Blues)

However, it is worth considering that Rock alone covers 53.4% of the market which is greater than combined market (17%) of the chosen genres.

## Query to retrieve Total sales generated by each Employee

In [5]:
%%sql

With customer_sales AS
                    (SELECT c.customer_id, c.first_name ||" "|| c.last_name
                     customer_name, support_rep_id, SUM(total) total_purchases
                     FROM customer c JOIN invoice i ON c.customer_id=i.customer_id
                     GROUP BY 1 ORDER BY 3 DESC
                    ),
    employee_performance AS
                    (SELECT first_name ||" "|| last_name Employee_name,
                     hire_date "Hired On",
                     CURRENT_TIMESTAMP - birthdate Age,
                     title Title,
                     ROUND(SUM(total_purchases),2) "Total Sales"
                     FROM employee LEFT JOIN customer_sales ON 
                     employee_id = support_rep_id
                     GROUP BY 1 ORDER BY 5 DESC
                     
                    )
    
SELECT * FROM employee_performance; 


Done.


Employee_name,Hired On,Age,Title,Total Sales
Jane Peacock,2017-04-01 00:00:00,49,Sales Support Agent,1731.51
Margaret Park,2017-05-03 00:00:00,75,Sales Support Agent,1584.0
Steve Johnson,2017-10-17 00:00:00,57,Sales Support Agent,1393.92
Andrew Adams,2016-08-14 00:00:00,60,General Manager,None
Laura Callahan,2017-03-04 00:00:00,54,IT Staff,None
Michael Mitchell,2016-10-17 00:00:00,49,IT Manager,None
Nancy Edwards,2016-05-01 00:00:00,64,Sales Manager,None
Robert King,2017-01-02 00:00:00,52,IT Staff,None


Although, at first insight all the employees are seemed to be performing as expected. The sales generated is positively co-related to time spent with company; however a deeper look shows that Steve was hired with more time gap than Margaret and yet his sales performance is very close to him.

## Query to retrieve Customer and Sales data by each Country

In [6]:
%%sql

DROP VIEW IF Exists DATA;

CREATE VIEW Data AS
            SELECT country Country,
             COUNT(distinct c.customer_id) "Total Customers",
             ROUND(SUM(total),2) "Total Sales",
             ROUND(SUM(total)/COUNT(c.customer_id), 2) "Average Sales per Customer"
             FROM customer c JOIN invoice i ON c.customer_id=i.customer_id
            GROUP BY Country;
            

SELECT
        country,
        "Total Customers",
        "Total Sales",
        "Average Sales per Customer"
FROM
(SELECT *, 0 sort FROM Data WHERE "Total Customers" > 1

UNION

SELECT "Others", SUM("Total Customers"),
SUM("Total Sales"),
AVG("Average Sales per Customer"),
1 sort
FROM Data
WHERE "Total Customers" = 1
ORDER BY sort, 2 DESC);



Done.
Done.
Done.


Country,Total Customers,Total Sales,Average Sales per Customer
USA,13,1040.49,7.94
Canada,8,535.59,7.05
Brazil,5,427.68,7.01
France,5,389.07,7.78
Germany,4,334.62,8.16
United Kingdom,3,245.52,8.77
Czech Republic,2,273.24,9.11
India,2,183.15,8.72
Portugal,2,185.13,6.38
Others,15,1094.94,7.444


## Query to find out if invoice were an album purchase

In [7]:
%%sql

DROP VIEW IF Exists Album_Purchase;

CREATE VIEW Album_Purchase AS
SELECT CASE
        WHEN (
             SELECT distinct t.track_id
             FROM track t NATURAL JOIN album a
             WHERE a.album_id LIKE ( SELECT distinct t.album_id
                                     FROM invoice i NATURAL JOIN invoice_line il
                                     NATURAL JOIN track t
                                     WHERE i.invoice_id = mi_id.invoice_id)
              EXCEPT
            
              SELECT t.track_id
              FROM invoice i NATURAL JOIN invoice_line il
              NATURAL JOIN track t
              WHERE i.invoice_id = mi_id.invoice_id
             ) IS NULL
        AND
            (
              SELECT t.track_id
              FROM invoice i NATURAL JOIN invoice_line il
              NATURAL JOIN track t
              WHERE i.invoice_id = mi_id.invoice_id
                
              EXCEPT  
              
             SELECT distinct t.track_id
             FROM track t NATURAL JOIN album a
             WHERE a.album_id LIKE ( SELECT distinct t.album_id
                                     FROM invoice i NATURAL JOIN invoice_line il
                                     NATURAL JOIN track t
                                     WHERE i.invoice_id = mi_id.invoice_id)  
            )IS NULL THEN "Yes"
        ELSE "No"
        END AS category
FROM invoice mi_id;

SELECT * FROM Album_Purchase LIMIT 10;

Done.
Done.
Done.


category
No
No
No
Yes
No
Yes
Yes
No
Yes
No


Number and Percentage of invoices as whole album purchase.

In [8]:
%%sql

SELECT category Category, COUNT(*) "Number of invoices",
ROUND(
    CAST(COUNT(*) AS FLOAT)
    /
    (SELECT COUNT(*) FROM Album_Purchase),2)*100 "% of invoices"
FROM Album_Purchase
Group BY 1;

Done.


Category,Number of invoices,% of invoices
No,500,81.0
Yes,114,19.0


From the analysis, only 19% of the sales involved full album. This shows that only a small percentage of market is willing to buy full albums therefore it would be best to not buy albums as whole. Chinook may focus on purchasing only the most popular tracks from every album.

# The END.